## Building an Amortization Table

In [37]:
import pandas as pd
import numpy as np
from datetime import date

In [38]:
Interest_Rate = 0.04
Years = 30
Payments_Year = 12
Principal = 200000
Add1_Principal = 50
start_date = (date(2016,1,1))

In [39]:
pmt = np.pmt(Interest_Rate/Payments_Year, Years * Payments_Year, Principal)

In [40]:
pmt

-954.83059093090765

This means we need to pay $954.83 every month. 

In [41]:
# Period to calculate
per = 1

# Calculate the interest
ipmt = np.ipmt(Interest_Rate/Payments_Year, per, Years * Payments_Year, Principal)

# Calculate the principal
ppmt = np.ppmt(Interest_Rate/Payments_Year, per, Years * Payments_Year, Principal)

print(ipmt,ppmt) 

-666.6666666666667 -288.163924264


In other words, the first payement of 984.83 is composed of \$666.67 as interest and only \$288.16 in principal.

Lets Look at the breakdown for period of 20 years or 240 months.

In [42]:
per = 240

# Calculate the interest.
ipmt = np.ipmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)

# Calculate the principal.
ppmt = np.ppmt(Interest_Rate/Payments_Year, per, Years*Payments_Year, Principal)
print(ipmt,ppmt)

-316.49041533656924 -638.340175594


In this case we are paying much more towards principal and much less towards the principal.

### Building the Table

In [43]:
rng = pd.date_range(start_date,periods= Years * Payments_Year, freq='MS')
rng.name = "Payment_Date"

In [44]:
df = pd.DataFrame(index=rng,columns = ['Payment', 'Principal', 'Interest', 'Addl_Principal'], dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Period"

In [45]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_Principal
Period,,,,,
1,2016-01-01,NaN,NaN,NaN,NaN
2,2016-02-01,NaN,NaN,NaN,NaN
3,2016-03-01,NaN,NaN,NaN,NaN
4,2016-04-01,NaN,NaN,NaN,NaN
5,2016-05-01,NaN,NaN,NaN,NaN


In [46]:
df["Principal"] = np.ppmt(Interest_Rate/Payments_Year, df.index, Years*Payments_Year, Principal)
df["Interest"] = np.ipmt(Interest_Rate/Payments_Year, df.index, Years*Payments_Year, Principal)
df["Payment"] = np.pmt(Interest_Rate/Payments_Year, Years*Payments_Year, Principal)

In [47]:
df["Addl_Principal"] = -Add1_Principal
df = df.round(2)

In [48]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_Principal
Period,,,,,
1,2016-01-01,-954.83,-288.16,-666.67,-50
2,2016-02-01,-954.83,-289.12,-665.71,-50
3,2016-03-01,-954.83,-290.09,-664.74,-50
4,2016-04-01,-954.83,-291.06,-663.78,-50
5,2016-05-01,-954.83,-292.03,-662.81,-50


In [49]:
df["Cumulative_Principal"] = (df["Principal"] + df["Addl_Principal"]).cumsum()


In [50]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_Principal,Cumulative_Principal
Period,,,,,,
1,2016-01-01,-954.83,-288.16,-666.67,-50,-338.16
2,2016-02-01,-954.83,-289.12,-665.71,-50,-677.28
3,2016-03-01,-954.83,-290.09,-664.74,-50,-1017.37
4,2016-04-01,-954.83,-291.06,-663.78,-50,-1358.43
5,2016-05-01,-954.83,-292.03,-662.81,-50,-1700.46


In [51]:
df["Cumulative_Principal"] = df["Cumulative_Principal"].clip(lower=-Principal)

In [52]:
df["Curr_Balance"] = Principal + df["Cumulative_Principal"]

In [53]:
last_payment = df.query("Curr_Balance <= 0")["Curr_Balance"].idxmax(axis=1, skipna=True)
df.ix[last_payment]

Payment_Date            2044-06-01 00:00:00
Payment                             -954.83
Principal                           -896.33
Interest                              -58.5
Addl_Principal                          -50
Cumulative_Principal                -200000
Curr_Balance                              0
Name: 342, dtype: object